In [2]:
import pandas
import glob
import os

In [3]:
def load_input_metadata(metadata : str) -> pandas.DataFrame:
    input_metadata = pandas.read_csv(metadata)
    return input_metadata

In [4]:
print(os.getcwd())

/Users/mf019/bioinformatics/longread_GWAS/notebooks/OspC_typing


In [125]:

metadata = "../metadata/longread_metadata_v2.csv"

input_metadata = load_input_metadata(metadata)
known_types = input_metadata[['Isolate', 'OspC_Type']]
print(known_types)

             Isolate OspC_Type
0    5A18NP1-JBb08-A       NaN
1             UNY147         J
2    5A18NP1-JBb08-C       NaN
3    5A18NP1-JBb08-B       NaN
4    5A18NP1-JBb08-E       NaN
..               ...       ...
374              K60         A
375              K82         K
376              K11         K
377              K59         K
378              K69         A

[379 rows x 2 columns]


## Next Steps
Ok, so now what we'll do is take the table above, pull all isolates that have a known ospc_type, then pull the ospc annotations from each isolate, then make a tree, then confirm tree layout with known types, then run tree of all known ospc annotations, then use previous labels as classifications for each cluster.

> What are the characteristics that differentiate each OspC type.
>> Can I use motif scanning? Definitely MSA.

# Okay, Ira has sent me a large file of OspC types and AA seqs
I will be taking this, making a blast DB and then I need to isolate the OspC seqs from each assembly, then blast against the OspC database.

In [6]:

ospc_genotype_to_ref = {
     'A' : 'X69596',
   'AT1' : 'EU482041',
   'AT2' : 'EU482042',
   'AT3' : 'EU482043',
   'AT4' : 'EU482044',
   'AT5' : 'EU482045',
   'AT6' : 'EU482046',
   'AT7' : 'EU482047',
   'AT8' : 'EU482048',
   'AT9' : 'EU482049',
  'AT10' : 'EU482050',
  'AT11' : 'EU482051',
  'AT12' : 'EU482052',
  'AT13' : 'EU482053',
  'AT14' : 'EU482054',
  'AT15' : 'EU482055',
  'AT16' : 'EU482056',
    'Ba' : 'EF537413',
    'Bb' : 'NC_011724',
     'C' : 'DQ437462',
    'Da' : 'AF029863',
    'Db' : 'GQ478283',
     'E' : 'AY275221',
    'Fa' : 'AY275225',
    'Fb' : 'EF537433',
    'Fc' : 'GQ478285',
     'G' : 'AY275223',
    'Ha' : 'EU377781',
    'Hb' : 'GQ478286',
    'Ia' : 'AY275219',
    'Ib' : 'EU377752',
     'J' : 'CP001535',
     'K' : 'AY275214',
     'L' : 'EU375832',
     'M' : 'CP001550',
     'N' : 'EU377775',
     'O' : 'FJ997281',
     'T' : 'AY275222',
    'Ua' : 'EU377769',
    'Ub' : 'GQ478287',
    'A3' : 'EF592541',
    'B3' : 'EF592542',
    'C3' : 'EF592543',
    'D3' : 'EF592544',
    'E3' : 'EF592545',
    'F3' : 'EF592547',
    'H3' : 'FJ932733',
    'I3' : 'FJ932734',
    'B.bissettii_25015' : 'U04282'}
len(ospc_genotype_to_ref)

49

In [7]:
# ok now to pull all of those seqs via entrez direct and then make a multifasta file upon which to build a blastdb from
# we will use the following command to pull the sequences from the NCBI nucleotide database using our defined dictionary
# and we will also call the command using the biopython wrapper for entrez direct

#for ospc_type, ref in ospc_genotype_to_ref.items():
#    print(f"esearch -db nucleotide -query {ref} | efetch -format fasta > ospc_seqs/{ospc_type}.fasta")
#
from Bio import Entrez
Entrez.email = "mfoster11@mgh.harvard.edu"
#for ospc_type, ref in ospc_genotype_to_ref.items():
#    print(f"fetching record for {ospc_type} : {ref}")
#    handle = Entrez.efetch(db="nucleotide", id=ref, rettype="fasta", retmode="text")
#    record = handle.read()
#    with open(f"ospc_seqs/{ospc_type}.fasta", "w") as f:
#        print("writing record to file!")
#        f.write(record[0::])
#    handle.close()
#print("done!")

In [8]:
# ok so some of those entries include the entire plasmid sequence not just the ospc gene, so we will need to extract the ospc gene from the plasmid sequence
# the ones that are plasmid sequences are as follows:
# Bb, J, M,
# we will need to extract the ospc gene from these sequences

#for ospc_type in ['Bb', 'J', 'M']:
#    print(ospc_genotype_to_ref[ospc_type])
#
#coords = {
#    'Bb' : (16904,17540),
#    'J' : (16909,17545),
#    'M' : (16916,17555)
#}
#
## ok we already have the full plasmid seqs for each of these three types, we also now have the coordinates
from Bio import SeqIO
#
#for ospc_type in ['Bb', 'J', 'M']:
#    print(f"extracting ospc gene from {ospc_type} sequence")
#    with open(f"ospc_seqs/{ospc_type}.fasta", "r") as f:
#        record = SeqIO.read(f, "fasta")
#        print(record)
#        ospc_gene = record[coords[ospc_type][0]:coords[ospc_type][1]]
#        with open(f"ospc_seqs/{ospc_type}_ospc.fasta", "w") as f:
#            SeqIO.write(ospc_gene, f, "fasta")

In [116]:
import glob

ospc_seqs = glob.glob("ospc_seqs/*.fasta")
print(ospc_seqs)
ospc_seq_file = []
for seq in ospc_seqs:
    record = SeqIO.read(seq, "fasta")
    record.id = "OspC_Type-" + seq.split("/")[-1].split(".")[0]
    record.description = ""
    print(record.id)
    ospc_seq_file.append(record)
with open("all_ospc.fasta", "w") as f:
    SeqIO.write(ospc_seq_file, f, "fasta")


AT_seqs = glob.glob("AT_seqs/*.fasta")
print(AT_seqs)
AT_seq_file = []
for seq in AT_seqs:
    record = SeqIO.read(seq, "fasta")
    record.id = "OspC_Type-" + seq.split("/")[-1].split(".")[0]
    record.description = ""
    print(record.id)
    AT_seq_file.append(record)
with open("all_ospc_ATs.fasta", "w") as f:
    SeqIO.write(AT_seq_file, f, "fasta")

['ospc_seqs/A.fasta', 'ospc_seqs/I3.fasta', 'ospc_seqs/E3.fasta', 'ospc_seqs/Fa.fasta', 'ospc_seqs/Fc.fasta', 'ospc_seqs/C.fasta', 'ospc_seqs/G.fasta', 'ospc_seqs/Bb.fasta', 'ospc_seqs/Da.fasta', 'ospc_seqs/E.fasta', 'ospc_seqs/Ha.fasta', 'ospc_seqs/Fb.fasta', 'ospc_seqs/C3.fasta', 'ospc_seqs/Ba.fasta', 'ospc_seqs/A3.fasta', 'ospc_seqs/Db.fasta', 'ospc_seqs/Hb.fasta', 'ospc_seqs/J.fasta', 'ospc_seqs/Ub.fasta', 'ospc_seqs/H3.fasta', 'ospc_seqs/D3.fasta', 'ospc_seqs/F3.fasta', 'ospc_seqs/Ia.fasta', 'ospc_seqs/L.fasta', 'ospc_seqs/N.fasta', 'ospc_seqs/Ua.fasta', 'ospc_seqs/B3.fasta', 'ospc_seqs/K.fasta', 'ospc_seqs/O.fasta', 'ospc_seqs/T.fasta', 'ospc_seqs/Ib.fasta', 'ospc_seqs/M.fasta']
OspC_Type-A
OspC_Type-I3
OspC_Type-E3
OspC_Type-Fa
OspC_Type-Fc
OspC_Type-C
OspC_Type-G
OspC_Type-Bb
OspC_Type-Da
OspC_Type-E
OspC_Type-Ha
OspC_Type-Fb
OspC_Type-C3
OspC_Type-Ba
OspC_Type-A3
OspC_Type-Db
OspC_Type-Hb
OspC_Type-J
OspC_Type-Ub
OspC_Type-H3
OspC_Type-D3
OspC_Type-F3
OspC_Type-Ia
OspC_Type-L


In [117]:
# ok now lets make a local blast DB using biopython
from Bio.Blast.Applications import NcbimakeblastdbCommandline
cmd = NcbimakeblastdbCommandline(input_file="all_ospc.fasta", dbtype="nucl", out="blastdb/ospc/ospc")
print(cmd)
stdout, stderr = cmd()
print(stdout)
print(stderr)

makeblastdb -out blastdb/ospc/ospc -dbtype nucl -in all_ospc.fasta


Building a new DB, current time: 03/04/2024 14:53:27
New DB name:   /Users/mf019/bioinformatics/longread_GWAS/notebooks/OspC_typing/blastdb/ospc/ospc
New DB title:  all_ospc.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 32 sequences in 0.018918 seconds.






In [118]:
# ok now lets make a local blast DB using biopython
from Bio.Blast.Applications import NcbimakeblastdbCommandline
cmd = NcbimakeblastdbCommandline(input_file="all_ospc_ATs.fasta", dbtype="nucl", out="blastdb/ospcAT/ospcAT")
print(cmd)
stdout, stderr = cmd()
print(stdout)
print(stderr)

makeblastdb -out blastdb/ospcAT/ospcAT -dbtype nucl -in all_ospc_ATs.fasta


Building a new DB, current time: 03/04/2024 14:53:32
New DB name:   /Users/mf019/bioinformatics/longread_GWAS/notebooks/OspC_typing/blastdb/ospcAT/ospcAT
New DB title:  all_ospc_ATs.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 17 sequences in 0.00470495 seconds.






In [220]:
# ok now lets pull all of the ospCs from all assemblies and then blast them against our local blastdb!
# we will use the following command to pull the sequences from the NCBI nucleotide database using our defined dictionary
print(os.getcwd())
assemblies = glob.glob("../../assemblies/*/annotation/*/*.gbff")
print(assemblies)

ospc_identifiers = [
    "ospC",
    "OspC",
    "Ospc",
    "ospc",
    "outer surface protein C",
    "Outer surface protein C",
    "outer surface lipoprotein C",
    "Outer surface lipoprotein C",
    "outer surface protein c",
]

/Users/mf019/bioinformatics/longread_GWAS/notebooks/OspC_typing
['../../assemblies/pacbio/annotation/MR616/MR616.gbff', '../../assemblies/pacbio/annotation/B356/B356.gbff', '../../assemblies/pacbio/annotation/B500/B500.gbff', '../../assemblies/pacbio/annotation/MC123/MC123.gbff', '../../assemblies/pacbio/annotation/B247/B247.gbff', '../../assemblies/pacbio/annotation/B418/B418.gbff', '../../assemblies/pacbio/annotation/MC149/MC149.gbff', '../../assemblies/pacbio/annotation/MR614/MR614.gbff', '../../assemblies/pacbio/annotation/MC104/MC104.gbff', '../../assemblies/pacbio/annotation/MC105/MC105.gbff', '../../assemblies/pacbio/annotation/MR641/MR641.gbff', '../../assemblies/pacbio/annotation/BL224/BL224.gbff', '../../assemblies/pacbio/annotation/B348/B348.gbff', '../../assemblies/pacbio/annotation/B331/B331.gbff', '../../assemblies/pacbio/annotation/BL206/BL206.gbff', '../../assemblies/pacbio/annotation/BL522/BL522.gbff', '../../assemblies/pacbio/annotation/B483/B483.gbff', '../../assembl

In [280]:
ospc_genes = {}
for file in assemblies:
    #print(file)
    sample_id = file.split("/")[-1].split(".")[0].replace("_200","" )
    ospc_genes[sample_id] = []
    with open(file, "r") as f:
        record = SeqIO.parse(f, "genbank")
        for rec in record:
            #print(rec.id)
            for feature in rec.features:
                if feature.type == "CDS":
                    product = feature.qualifiers['product']
                    if any(x in product[0] for x in ospc_identifiers):
                        #feature.qualifiers['product']
                        # get sequence
                        #print(feature.location)
                        seq = feature.extract(rec.seq)
                        gene = SeqIO.SeqRecord(seq, id=product[0], name=product[0], description=f"gene: {product[0]} from isolate {sample_id}", dbxrefs=None)
                        ospc_genes[sample_id].append(gene)

In [281]:
import pprint
pprint.pprint(ospc_genes)

{'5A18NP1-JBb08-A': [SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='outer surface protein OspC', name='outer surface protein OspC', description='gene: outer surface protein OspC from isolate 5A18NP1-JBb08-A', dbxrefs=[]),
                     SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='outer surface protein OspC', name='outer surface protein OspC', description='gene: outer surface protein OspC from isolate 5A18NP1-JBb08-A', dbxrefs=[])],
 '5A18NP1-JBb08-B': [SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='outer surface protein OspC', name='outer surface protein OspC', description='gene: outer surface protein OspC from isolate 5A18NP1-JBb08-B', dbxrefs=[]),
                     SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='outer surface protein OspC', name='outer surface protein OspC', description='gene: outer surface protein 

In [297]:
for sample in ospc_genes:
    print(sample)
    print(ospc_genes[sample])
    with open(f"temp_in/{sample}-osps.fasta", "w") as f:
        SeqIO.write(ospc_genes[sample], f"temp_in/{sample}-osps.fasta", "fasta")

MR616
[SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='Arthropod-specific outer surface protein C', name='Arthropod-specific outer surface protein C', description='gene: Arthropod-specific outer surface protein C from isolate MR616', dbxrefs=[])]
B356
[SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='Outer surface protein C', name='Outer surface protein C', description='gene: Outer surface protein C from isolate B356', dbxrefs=[])]
B500
[SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='Major outer surface protein C', name='Major outer surface protein C', description='gene: Major outer surface protein C from isolate B500', dbxrefs=[])]
MC123
[SeqRecord(seq=Seq('ATGAAAAAGAATACATTAAGTGCAATATTAATGACTTTATTTTTATTTATATCT...TAA'), id='Mammal-specific outer surface protein C', name='Mammal-specific outer surface protein C', description='gene: Mammal-specific outer surface protein C

In [299]:
# ok now we can blast each file against our local blastdb
from Bio.Blast.Applications import NcbiblastnCommandline
for sample in ospc_genes:
    print(f"blasting {sample}")
    cmd = NcbiblastnCommandline(query=f"temp_in/{sample}_osps.fasta", db="blastdb/ospc/ospc", out=f"temp_out/{sample}-osps_blast.xml", outfmt=5)
    print(cmd)
    stdout, stderr = cmd()
    print(stdout)
    print(stderr)

blasting MR616
blastn -out temp_out/MR616-osps_blast.xml -outfmt 5 -query temp_in/MR616_osps.fasta -db blastdb/ospc/ospc


blasting B356
blastn -out temp_out/B356-osps_blast.xml -outfmt 5 -query temp_in/B356_osps.fasta -db blastdb/ospc/ospc


blasting B500
blastn -out temp_out/B500-osps_blast.xml -outfmt 5 -query temp_in/B500_osps.fasta -db blastdb/ospc/ospc


blasting MC123
blastn -out temp_out/MC123-osps_blast.xml -outfmt 5 -query temp_in/MC123_osps.fasta -db blastdb/ospc/ospc


blasting B247
blastn -out temp_out/B247-osps_blast.xml -outfmt 5 -query temp_in/B247_osps.fasta -db blastdb/ospc/ospc


blasting B418
blastn -out temp_out/B418-osps_blast.xml -outfmt 5 -query temp_in/B418_osps.fasta -db blastdb/ospc/ospc


blasting MC149
blastn -out temp_out/MC149-osps_blast.xml -outfmt 5 -query temp_in/MC149_osps.fasta -db blastdb/ospc/ospc


blasting MR614
blastn -out temp_out/MR614-osps_blast.xml -outfmt 5 -query temp_in/MR614_osps.fasta -db blastdb/ospc/ospc


blasting MC104
blastn -out t

In [300]:
# ok now we can blast each file against our local blastdb
from Bio.Blast.Applications import NcbiblastnCommandline
for sample in ospc_genes:
    print(f"blasting {sample}")
    cmd = NcbiblastnCommandline(query=f"temp_in/{sample}_osps.fasta", db="blastdb/ospcAT/ospcAT", out=f"temp_out/{sample}-ospsAT_blast.xml", outfmt=5)
    print(cmd)
    stdout, stderr = cmd()
    print(stdout)
    print(stderr)

blasting MR616
blastn -out temp_out/MR616-ospsAT_blast.xml -outfmt 5 -query temp_in/MR616_osps.fasta -db blastdb/ospcAT/ospcAT


blasting B356
blastn -out temp_out/B356-ospsAT_blast.xml -outfmt 5 -query temp_in/B356_osps.fasta -db blastdb/ospcAT/ospcAT


blasting B500
blastn -out temp_out/B500-ospsAT_blast.xml -outfmt 5 -query temp_in/B500_osps.fasta -db blastdb/ospcAT/ospcAT


blasting MC123
blastn -out temp_out/MC123-ospsAT_blast.xml -outfmt 5 -query temp_in/MC123_osps.fasta -db blastdb/ospcAT/ospcAT


blasting B247
blastn -out temp_out/B247-ospsAT_blast.xml -outfmt 5 -query temp_in/B247_osps.fasta -db blastdb/ospcAT/ospcAT


blasting B418
blastn -out temp_out/B418-ospsAT_blast.xml -outfmt 5 -query temp_in/B418_osps.fasta -db blastdb/ospcAT/ospcAT


blasting MC149
blastn -out temp_out/MC149-ospsAT_blast.xml -outfmt 5 -query temp_in/MC149_osps.fasta -db blastdb/ospcAT/ospcAT


blasting MR614
blastn -out temp_out/MR614-ospsAT_blast.xml -outfmt 5 -query temp_in/MR614_osps.fasta -db blas

In [301]:
# ok now we can parse the blast results and then add the ospc type to the metadata
from Bio.Blast import NCBIXML
import os

def parse_blast_results(blast_results : str) -> dict:
    file_size = os.path.getsize(blast_results)
    if file_size == 0:
        return None
    results = {}
    with open(blast_results, "r") as f:
        blast_records = NCBIXML.parse(f)
        for record in blast_records:
            results[record.query] = []
            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    results[record.query].append((alignment.title, hsp.expect, hsp.score,))
    return results

In [313]:
from numpy import empty


checked_types_ospc = pandas.DataFrame(columns=["Isolate", "ospc_newtype"])
blast_results_ospc = glob.glob("temp_out/*-osps_blast.xml")

for result in blast_results_ospc:
    sample_id = result.split("/")[-1].split(".")[0].replace("-osps_blast", "")
    isolate = sample_id
    ospc_type = "unknown"
    parsed_results = parse_blast_results(result)
    if parsed_results is not None:
        for result in parsed_results:
            if len(parsed_results[result]) > 0:
                best_hit = parsed_results[result][0][0]
                coords = parsed_results[result][0][0].find("OspC_Type-")
                ospc_type = parsed_results[result][0][0][coords+10:coords+14]
                print(isolate, ospc_type)
                checked_types_ospc.loc[len(checked_types_ospc)] = [isolate, ospc_type]
                break
    else:
        print(isolate, ospc_type)
        checked_types_ospc.loc[len(checked_types_ospc)] = [isolate, "unknown"]


URI101 A
URI114 Ia
UWI249 M
UNY133 Fa
UNY209 Ia
ESI316 L
ESI303 Bb
URI71 K
URI64 K
UWI238 L
UNY142 G
UNY157 Bb
ESI286 Bb
URI79 Bb
UWI225 Fb
UWI230 Ub
K57 A
UNY201 J
UNY214 E3
UWI241 G
UWI254 Hb
UCT106 N
UCT113 Hb
URI86 K
URI93 A
UNY134 N
MR616 G
UCT109 A
UCT76 K
URI89 A
UNY150 Bb
UNY145 A
ESI289 Bb
K58 A
URI63 A
ESI304 L
ESI311 L
L113 L
UNY213 Fa
UNY206 N
ESI319 L
UWI237 E
UWI222 Hb
UNY158 G
ESI294 Bb
URI94 K
URI81 N
K3 A
B31-K2 A
UWI253 Hb
UWI246 Fa
K21 E
UNY129 A
ESI295 L
UNY159 Ub
UWI223 K
UWI236 G
ESI318 Bb
UNY207 unknown
UNY212 K
UWI247 Hb
UWI252 L
K2 A
UCT115 Hb
URI88 Hb
UCT62 K
URI107 A
URI112 K
UNY135 Ia
BL224 K
ESI305 Bb
K59 K
URI77 K
UNY144 Ub
UNY151 Hb
UNY215 G
UNY200 A
UWI231 E3
UWI224 Hb
UCT92 K
URI78 Bb
K56 K
ESI287 Bb
ESI292 Bb
URI87 A
URI108 Bb
UWI255 Ia
UWI240 M
BL082_H1 A
UNY132 Ia
UWI248 Hb
URI100 Bb
UNY156 K
UNY143 M
UWI239 O
URI65 A
URI70 Bb
ESI317 Bb
UNY208 Hb
UWI257 B3
UWI242 A
K30 Ia
K25 Ia
UNY138 N
K7 A
URI90 N
URI85 N
UCT110 K
UWI233 G
UWI226 O
UNY149 K
ESI285

In [314]:
checked_types_ospc.to_csv("new_ospC_types.csv", index=False)

In [315]:
print(len(checked_types_ospc))
#checked_types_ospc.drop_duplicates(subset="Isolate", keep="first", inplace=True)
pprint.pprint(checked_types_ospc)
duplicates = checked_types_ospc[checked_types_ospc['Isolate'].duplicated()]
print(duplicates)



379
    Isolate ospc_newtype
0    URI101            A
1    URI114           Ia
2    UWI249            M
3    UNY133           Fa
4    UNY209           Ia
..      ...          ...
374  UNY181            K
375  UNY194            K
376     K12            K
377  UWI275            M
378  UWI260           F3

[379 rows x 2 columns]
Empty DataFrame
Columns: [Isolate, ospc_newtype]
Index: []


In [316]:
checked_types_AT = pandas.DataFrame(columns=["Isolate", "AT_type"])
blast_results_AT = glob.glob("temp_out/*-ospsAT_blast.xml")

for result in blast_results_AT:
    #print(result)
    sample_id = result.split("/")[-1].replace("-ospsAT_blast.xml", "")
    isolate = sample_id
    parsed_results = parse_blast_results(result)
    if parsed_results is not None:
        for result in parsed_results:
            if len(parsed_results[result]) > 0:
                best_hit = parsed_results[result][0][0]
                coords = parsed_results[result][0][0].find("OspC_Type-")
                ospc_type = parsed_results[result][0][0][coords+10:coords+14]
                print(isolate, ospc_type)
                checked_types_AT.loc[len(checked_types_AT)] = [isolate, ospc_type]
                break
    else:
        print(isolate, ospc_type)
        checked_types_AT.loc[len(checked_types_AT)] = [isolate, "unknown"]


B31-5A4 AT1
UMA2 AT6
UNY182 AT11
UWI239 Bbis
UNY164 AT8
URI43 AT1
UWI262 AT1
UNY205 AT9
B348 AT5
5A18NP1-JBb08-C AT1
ESI308 AT2
URI122 AT11
ESI317 AT2
URI103 AT11
EU14_B89613 AT2
K72 AT11
URI84 AT11
UWI243 AT11
K17 AT11
UCT109 AT1
URI39 AT1
UWI226 Bbis
UNY145 AT1
UNY134 AT13
URI90 AT13
UWI269 AT8
UWI257 Bbis
URI48 AT13
EU51_B192411 Bbis
UCT30 AT11
K58 AT1
K66 AT1
K80 Bbis
BL206 AT1
URI117 AT11
ESI303 AT2
UMA9 AT2
ESI287 AT2
MR641 AT8
UNY151 AT8
UWI232 AT11
UNY189 AT9
UCT74 AT1
K22 AT1
UCT92 AT11
UNY170 AT1
UNY196 AT9
K1 AT1
ESI28 Bbis
URI108 AT2
ESI322 Bbis
UNY211 AT11
UWI276 AT13
UWI248 AT8
URI57 AT2
HB-19 AT9
UNY174 AT2
UWI229 AT13
URI36 AT9
UNY192 AT1
UCT106 AT13
K18 AT11
UCT96 AT2
UWI272 AT1
URI53 AT9
BB-8 AT10
UNY215 AT7
ESI318 AT2
ESI307 AT2
K62 AT8
K84 AT2
UNY130 AT11
UWI253 AT8
URI94 AT11
URI72 AT1
UCT51 AT9
UNY155 AT11
UWI236 AT7
URI17 AT2
URI58 AT9
UWI247 AT8
UWI279 AT6
K90 AT2
K76 AT8
ESI313 AT2
URI107 AT1
ESI297 AT2
UNY199 AT5
UWI222 AT8
UNY141 AT6
UMA13 AT6
K13 AT11
URI22 

In [317]:
pprint.pprint(checked_types_AT)

     Isolate AT_type
0    B31-5A4     AT1
1       UMA2     AT6
2     UNY182    AT11
3     UWI239    Bbis
4     UNY164     AT8
..       ...     ...
374   UNY209     AT9
375   ESI304    Bbis
376   UWI235    AT15
377   UNY156    AT11
378   UNY168     AT8

[379 rows x 2 columns]


In [318]:
pprint.pprint(known_types)
pprint.pprint(checked_types_ospc)
pprint.pprint(checked_types_AT)

             Isolate OspC_Type
0    5A18NP1-JBb08-A       NaN
1             UNY147         J
2    5A18NP1-JBb08-C       NaN
3    5A18NP1-JBb08-B       NaN
4    5A18NP1-JBb08-E       NaN
..               ...       ...
374              K60         A
375              K82         K
376              K11         K
377              K59         K
378              K69         A

[379 rows x 2 columns]
    Isolate ospc_newtype
0    URI101            A
1    URI114           Ia
2    UWI249            M
3    UNY133           Fa
4    UNY209           Ia
..      ...          ...
374  UNY181            K
375  UNY194            K
376     K12            K
377  UWI275            M
378  UWI260           F3

[379 rows x 2 columns]
     Isolate AT_type
0    B31-5A4     AT1
1       UMA2     AT6
2     UNY182    AT11
3     UWI239    Bbis
4     UNY164     AT8
..       ...     ...
374   UNY209     AT9
375   ESI304    Bbis
376   UWI235    AT15
377   UNY156    AT11
378   UNY168     AT8

[379 rows x 2 columns]


In [352]:
merged_table = []
# Merge the two tables on the 'Isolate' column
merged_table = pandas.merge(known_types, checked_types_ospc, on='Isolate', how='left')
print(merged_table)
merged_table = pandas.merge(merged_table, checked_types_AT, on='Isolate', how='left')
print(merged_table)
merged_table.to_csv("merged_ospC_types.csv", index=False)

             Isolate OspC_Type ospc_newtype
0    5A18NP1-JBb08-A       NaN            A
1             UNY147         J      unknown
2    5A18NP1-JBb08-C       NaN            A
3    5A18NP1-JBb08-B       NaN            A
4    5A18NP1-JBb08-E       NaN            A
..               ...       ...          ...
374              K60         A            A
375              K82         K            K
376              K11         K            K
377              K59         K            K
378              K69         A            A

[379 rows x 3 columns]
             Isolate OspC_Type ospc_newtype  AT_type
0    5A18NP1-JBb08-A       NaN            A      AT1
1             UNY147         J      unknown  unknown
2    5A18NP1-JBb08-C       NaN            A      AT1
3    5A18NP1-JBb08-B       NaN            A      AT1
4    5A18NP1-JBb08-E       NaN            A      AT1
..               ...       ...          ...      ...
374              K60         A            A      AT1
375              K82    

In [353]:
for i in range(0, len(merged_table)):
    if pandas.isnull(merged_table.at[i, 'OspC_Type']):
        merged_table.at[i, 'OspC_Type'] = merged_table.at[i, 'ospc_newtype']
merged_table.drop(columns=['ospc_newtype'], inplace=True)
print(merged_table)

             Isolate OspC_Type  AT_type
0    5A18NP1-JBb08-A         A      AT1
1             UNY147         J  unknown
2    5A18NP1-JBb08-C         A      AT1
3    5A18NP1-JBb08-B         A      AT1
4    5A18NP1-JBb08-E         A      AT1
..               ...       ...      ...
374              K60         A      AT1
375              K82         K     AT11
376              K11         K     AT11
377              K59         K     AT11
378              K69         A      AT1

[379 rows x 3 columns]


In [356]:
#input_metadata.drop(columns=['OspC_Type'], inplace=True)
output_metadata = pandas.merge(input_metadata, merged_table, on='Isolate', how='left')
print(output_metadata)
output_metadata.to_csv("longread_metadata_v3.csv", index=False)

             Isolate MLST  RST_Type   Source Origin Location skin/blood_pairs  \
0    5A18NP1-JBb08-A    1       NaN  culture     US       NY              NaN   
1             UNY147   34       3.0      NaN     US       NY              NaN   
2    5A18NP1-JBb08-C    1       NaN  culture     US       NY              NaN   
3    5A18NP1-JBb08-B    1       NaN  culture     US       NY              NaN   
4    5A18NP1-JBb08-E    1       NaN  culture     US       NY              NaN   
..               ...  ...       ...      ...    ...      ...              ...   
374              K60    1       1.0      NaN     US       RI              NaN   
375              K82    3       2.0      NaN     US       RI              NaN   
376              K11    3       2.0      NaN     US       CT              NaN   
377              K59    3       2.0      NaN     US       RI              NaN   
378              K69    1       1.0      NaN     US       CT              NaN   

    Disseminated    method 